<a href="https://colab.research.google.com/github/raqueeb/nlp_bangla/blob/master/transformers_bert_Bangla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## হাতেকলমে ন্যাচারাল ল্যাঙ্গুয়েজ প্রসেসিং: শুরুর ধারণা

[৭ম অধ্যায়](https://aiwithr.github.io/nlpbook/)

In [10]:
import os
os.environ['KAGGLE_USERNAME'] = "raqueeb" # username from the json file
os.environ['KAGGLE_KEY'] = "*****************"

In [11]:
!kaggle datasets files furcifer/bangla-newspaper-dataset

name                  size  creationDate         
--------------------  ----  -------------------  
data/data.json         4GB  2020-10-21 06:09:48  
data_v2/data_v2.json   2GB  2020-10-21 06:09:48  


In [12]:
!kaggle datasets download furcifer/bangla-newspaper-dataset -f data_v2/data_v2.json

 98% 436M/444M [00:17<00:00, 27.5MB/s]
100% 444M/444M [00:17<00:00, 26.2MB/s]


In [13]:
!unzip data_v2.json.zip

Archive:  data_v2.json.zip
  inflating: data_v2.json            


In [14]:
import json
with open('data_v2.json', encoding='utf-8') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['content'])
    labels.append(item['category'])

In [15]:
set(labels)

{'bangladesh',
 'economy',
 'education',
 'entertainment',
 'international',
 'life-style',
 'opinion',
 'sports',
 'technology'}

In [16]:
categories = list(set(labels))

In [17]:
categories

['international',
 'technology',
 'opinion',
 'sports',
 'economy',
 'bangladesh',
 'entertainment',
 'education',
 'life-style']

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
  sentences, labels, test_size=0.2, random_state=42)

In [19]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 136kB/s 
     |████████████████████████████████| 983kB 29.6MB/s 
     |████████████████████████████████| 266kB 41.0MB/s 
     |████████████████████████████████| 1.3MB 34.4MB/s 
     |████████████████████████████████| 1.1MB 37.7MB/s 
     |████████████████████████████████| 471kB 36.1MB/s 
     |████████████████████████████████| 2.9MB 33.3MB/s 
     |████████████████████████████████| 890kB 39.7MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.2-cp36-none-any.whl size=25276306 sha256=96b030c048538376b7bcf23ef12eaaec6896939164c7cb5b891e4ad259195c03
  Stored in directory: /root/.cache/pip/wheels/fe/56/00/25444c352cc843e5c5daea0e9517a192878ae22c2c6b5f4573
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=daa3aa21e7ffd34fad89c1c8b52f6011db5eedeaba20aeca1def38249025f9b5
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
  Created wheel for syn

In [20]:
import ktrain
from ktrain import text

In [21]:
MODEL_NAME = 'sagorsarker/bangla-bert-base'
t = text.Transformer(MODEL_NAME, maxlen=500)

In [ ]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 268
	95percentile : 669
	99percentile : 1079


In [ ]:
model = t.get_classifier()

In [ ]:
# ট্রেনিং করে নিচ্ছি
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [ ]:
%matplotlib inline
# learner.lr_find(show_plot=True, max_epochs=1)

In [ ]:
# preproc = t
# save model and Preprocessor instance after partially training
# ktrain.get_predictor(model, preproc).save('/tmp/my_predictor')

In [ ]:
# reload Predictor and extract model
# model = ktrain.load_predictor('/tmp/my_predictor').model

In [ ]:
learner.fit_onecycle(8e-5, 1)

In [ ]:
learner.validate(class_names=t.get_classes())

In [ ]:
# the one we got most wrong
learner.view_top_losses(n=1, preproc=t)

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
predictor.predict('বাংলাদেশে নদ-নদীগুলো ছড়িয়ে আছে জালের মতো। ঐতিহাসিকভাবেই এই অঞ্চলের নদ-নদীর গতি–প্রকৃতি ও প্রবাহ জটিল।')

In [ ]:
predictor.explain('শিক্ষা মন্ত্রণালয়ে এমন প্রস্তাব দিয়েছে মাউশি। মাউশির কর্মকর্তারা বলছেন, মন্ত্রণালয় অনুমোদন দিলেই সবাইকে সময়সহ অন্যান্য আনুষঙ্গিক বিষয় জানিয়ে দেওয়া হবে। তখন বেসরকারি বিদ্যালয়গুলোও কীভাবে নিজস্ব ব্যবস্থায় আবেদনপত্র বিতরণ ও লটারির কাজটি করবে, তা-ও জানিয়ে দেওয়া হবে। একই সঙ্গে ঢাকার বাইরের সরকারি বিদ্যালয়গুলোর বিষয়েও দিকনির্দেশনা দেওয়া হবে।')

In [ ]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# টেন্সরফ্লোতে মডেল সেভ করা
# model.save('/content/gdrive/My Drive/Colab Notebooks/saved_model/my_model')

In [ ]:
# predictor.save('/content/gdrive/My Drive/Colab Notebooks/saved_model/bangla_news_predictor')
# save model using transformers API after partially training
learner.model.save_pretrained('/content/gdrive/My Drive/Colab Notebooks/saved_model/bangla_news_predictor')

In [ ]:
# save Predictor (i.e., model and Preprocessor instance) after partially training
ktrain.get_predictor(model, preproc).save('/content/gdrive/My Drive/Colab Notebooks/saved_model/bangla_news_predictor_kt')

In [ ]:
# reload the model using transformers directly
# from transformers import *
# model = TFAutoModelForSequenceClassification.from_pretrained('/tmp/my_model')
# model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
reloaded_predictor = ktrain.load_predictor('/content/gdrive/My Drive/Colab Notebooks/saved_model/bangla_news_predictor')

In [ ]:
reloaded_predictor.predict_proba('বঙ্গবন্ধু টি–টোয়েন্টি কাপে কাল জেমকন খুলনার হয়ে খেলার ম্যাচে ৪ ওভারে ২৮ রানে ১ উইকেট নেন মাশরাফি বিন মুর্তজা।')

In [ ]:
reloaded_predictor.get_classes()